# Exercise - 12
Implement Batch Gradient Descent with early stopping for Softmax Regression without using Scikit-Learn, only NumPy. Use it on a classification task such as the iris dataset.

In [1]:
import numpy as np
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()

In [3]:
X, y = iris.data, iris.target

In [4]:
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [5]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

## Adding bias term

In [6]:
X_with_bias = np.column_stack((np.array([1] * len(X)), X))

In [7]:
X_with_bias[:5]

array([[1. , 5.1, 3.5, 1.4, 0.2],
       [1. , 4.9, 3. , 1.4, 0.2],
       [1. , 4.7, 3.2, 1.3, 0.2],
       [1. , 4.6, 3.1, 1.5, 0.2],
       [1. , 5. , 3.6, 1.4, 0.2]])

## Splitting dataset to train and test sets

In [8]:
def train_valid_test_split(
    X: np.ndarray, 
    y: np.ndarray, 
    *,
    ratio: float = 0.2, 
    random_state: int | None = None
) -> tuple[np.ndarray, ...]:     # represents multiple np.ndarray in tuple
    """
    Splits the dataset into X_train, X_valid, X_test, y_train, y_valid, y_test. Also shuffles the data.
    
    Parameters:
        - X (np.ndarray): NumPy ndarray of features.
        - y (np.ndarray): NumPy ndarray of labels / target.
        - ratio (float): Ratio of dataset to split into validation set and test set. Default 0.2, i.e. 20%-20% data is used for validation and test set.
        - random_state (int): Uses a specific seed for randomness if provided, default None.
    
    Returns:
        - tuple[np.ndarray, ...]: Tuple of 6 NumPy ndarray in order of X_train, X_valid, X_test, y_train, y_valid, y_test.
    """
    if random_state:
        np.random.seed(random_state)
        
    total_size = len(X)
    test_size = valid_size = int(total_size * ratio)
    train_size = total_size - test_size - valid_size
    
    rnd_indices = np.random.permutation(total_size)
    
    X_train = X[rnd_indices[:train_size]]
    y_train = y[rnd_indices[:train_size]]
    
    X_valid = X[rnd_indices[train_size: -test_size]]
    y_valid = y[rnd_indices[train_size: -test_size]]
    
    X_test = X[rnd_indices[-test_size:]]
    y_test = y[rnd_indices[-test_size:]]
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [9]:
X_train, X_valid, X_test, y_train, y_valid, y_test = train_valid_test_split(X_with_bias, y, random_state= 42)

In [10]:
list(map(len, (X_train, X_valid, X_test, y_train, y_valid, y_test)))

[90, 30, 30, 90, 30, 30]

## Transforming labels to one - hot vector

In [11]:
def one_hot_vector(y: np.ndarray) -> np.ndarray:
    return np.diag(np.ones(y.max() + 1))[y]

In [12]:
one_hot_vector(y_train[:10])

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [13]:
y_train_1_hot = one_hot_vector(y_train)
y_valid_1_hot = one_hot_vector(y_valid)
y_test_1_hot = one_hot_vector(y_test)

## Scaling Data

In [14]:
mean = X_train[:, 1:].mean(axis= 0)
std = X_train[:, 1:].std(axis= 0)

X_train[:, 1:] = (X_train[:, 1:] - mean) / std
X_valid[:, 1:] = (X_valid[:, 1:] - mean) / std
X_test[:, 1:] = (X_test[:, 1:] - mean) / std

In [15]:
X_train[0]

array([ 1.        ,  0.39652122, -0.65957023,  0.63935691,  0.10418645])

## Softmax Function
Implementing the softmax formula:

$\hat{\mathbf{p}_k} = \sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$

In [16]:
def softmax(logits: np.ndarray) -> np.ndarray:
    exps = np.exp(logits)
    exps_sum = np.sum(exps, axis= 1, keepdims= True)   # keepdims returns the result as array
    return exps / exps_sum

In [17]:
softmax(y_train_1_hot[:10])

array([[0.21194156, 0.57611688, 0.21194156],
       [0.57611688, 0.21194156, 0.21194156],
       [0.21194156, 0.21194156, 0.57611688],
       [0.21194156, 0.57611688, 0.21194156],
       [0.21194156, 0.57611688, 0.21194156],
       [0.57611688, 0.21194156, 0.21194156],
       [0.21194156, 0.57611688, 0.21194156],
       [0.21194156, 0.21194156, 0.57611688],
       [0.21194156, 0.57611688, 0.21194156],
       [0.21194156, 0.57611688, 0.21194156]])

## Training Model

In [18]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))

In [19]:
n_inputs, n_outputs

(5, 3)

In [24]:
eta = 0.1         # Learning rate
n_epochs = 50000  # number of epochs
m = len(X_train)  # number of training instances
epsilon = 1e-5    # adding this small value to remove nan values caused by pk hat = 0 while taking log
alpha = 0.01      # regularization hyperparameter
best_loss = float('inf')    # setting initial loss to infinity

np.random.seed(42)
theta = np.random.randn(n_inputs, n_outputs)    # random parameter initialization

for epoch in range(n_epochs):
    # getting scores/logits
    logits = X_train @ theta
    
    # using softmax function to get probabilities
    y_proba = softmax(logits)
    y_proba_valid = softmax(X_valid @ theta)
    
    # calculating cross entropy loss and l2_loss
    cross_entropy_loss = -(y_valid_1_hot * np.log(y_proba_valid + epsilon)).sum(axis=1).mean()    # J(theta) of softmax regression
    l2_loss = alpha / m * (theta[1:] ** 2).sum()     # l2 loss of ridge regression, to perform regularization
    total_loss = cross_entropy_loss + l2_loss
    
    # Early Stopping
    if total_loss < best_loss:
        best_loss = total_loss
    else:
        print(epoch - 1, best_loss)
        print(epoch, total_loss, "early stopping!")
        break
        
    # basic training part, calculating gradients and updating theta
    error = y_proba - y_train_1_hot
    gradients = 1 / m * X_train.T @ error
    theta = theta - eta * gradients

12449 0.10838445296778279
12450 0.10838445296914534 early stopping!


In [25]:
theta

array([[ 0.90418494,  5.17526854, -5.07331508],
       [-2.13513884,  1.95479478,  1.23508358],
       [ 3.24204237,  0.05690417, -1.42177338],
       [-4.37796656, -1.27985305,  5.2712322 ],
       [-4.61821162, -2.92627525,  4.14825106]])

In [26]:
logits = X_valid @ theta
y_proba = softmax(logits)
predictions = y_proba.argmax(axis= 1)

accuracy_score = (predictions == y_valid).mean()
print(f'{accuracy_score = :.2%}')

accuracy_score = 96.67%


In [27]:
logits = X_test @ theta
y_proba = softmax(logits)
predictions = y_proba.argmax(axis= 1)

accuracy_score = (predictions == y_test).mean()
print(f'{accuracy_score = :.2%}')

accuracy_score = 100.00%


Well it was kinda unexpected to get such an insane accuracy, but probably it was because of small size of dataset.